In [86]:
import tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np

In [87]:
## Load the trained model, scaler pickle and onehot encoder
model = load_model('model.h5')

## Load onehot encoder and scaler
with open ('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open ('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo= pickle.load(file)

with open ('scaler.pkl','rb') as file:
    scaler= pickle.load(file)      
    

In [88]:
# Example input data
input_data = {
    'CreditScore': 800,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 50,
    'Tenure': 3,
    'Balance': 6000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 40000
}

In [89]:
## Use OneHot Encoded 'Geography'
geo_encoded= onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\faroo\OneDrive\Desktop\ANN project\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [90]:
## Convert input data into dataframe 
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,Male,50,3,6000,2,1,1,40000


In [91]:
## Encode Gender categorical variable to numerical variable
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,1,50,3,6000,2,1,1,40000


In [92]:
## Concatenate encoded columns with input data dataframe
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,50,3,6000,2,1,1,40000,1.0,0.0,0.0


In [93]:
## Scaling the data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.53956105,  0.91324755,  1.05551796, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -1.05069891,  1.00150113,
        -0.57946723, -0.57638802]])

In [94]:
## Predict Churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 288ms/step


array([[0.02688791]], dtype=float32)

In [95]:
prediction_prob=prediction[0][0]
prediction_prob

0.026887909

In [96]:
if prediction_prob>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")
    


The customer is not likely to churn
